In [9]:
from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import random
import statistics
from IPython.utils import io
from tqdm.notebook import tqdm
# import sdl2.ext

import models
import utils

In [10]:
# %matplotlib inline
# %matplotlib ipympl
# %matplotlib tk
plt.rc('animation', html='jshtml')
plt.rcParams["image.cmap"] = "bone"

In [11]:
# in_data = tonic.datasets.NMNIST(save_to="../data", train=True) # (34, 34, 2) 3
# in_data = tonic.datasets.POKERDVS(save_to="../data",train=True) # (35, 35, 2) 3
in_data = tonic.datasets.DVSGesture(save_to="../data",train=True) # (128, 128, 2) 2
# in_data = tonic.datasets.ASLDVS(save_to="../data", train=True) # (240, 180, 2) 3

in_data.sensor_size

(128, 128, 2)

In [12]:
n_evs = 2991
noise = tonic.transforms.UniformNoise(
    sensor_size=in_data.sensor_size[:-1]+(1,),
    # n=int(n_evs*2.33333),
    # n=int(n_evs),
    # n=int(n_evs*0.42855),
    n=int(n_evs/9),
)
# to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), time_window=1000)
to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), n_time_bins=200)

In [13]:
net = models.Conv_FastLIF(models.kernels.gaussian5,0.9,1.5)#.cuda(0)
# net = models.Conv_LIF(models.kernels.gaussian5)#.cuda(0)
# net = models.FastLIF(0.85,3)
# net = models.LIF()
# d=models.Denoise(100)

In [14]:
random.seed(0)
ids = random.sample(range(len(in_data)), 1)
# ids = range(len(in_data))
print(ids)
for id in tqdm(ids):
    events, label = in_data[id]
    events = events.squeeze()
    events = np.array(list(filter(lambda ev: ev[2] == 0, events)))
    # print(events)

    true_frames = torch.tensor(to_frame(events)).float()  # .cuda(0)

    with torch.inference_mode():
        out_frames = net(true_frames)

    out_frames = utils.frame_merge(out_frames, 10).clamp_max(1).cpu()
    true_frames = utils.frame_merge(true_frames, 10).clamp_max(1).cpu()



[788]


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
with io.capture_output() as captured:
    anim = tonic.utils.plot_animation(np.concatenate((true_frames,out_frames),axis=3))
display(anim)

In [16]:
# anim.save("../img/lif-temp.gif",dpi=300)